📒 [NB 1-3] 파생 변수 생성 (Feature Creation)
1. 실습 데이터 준비
키, 몸무게, 입/퇴원 날짜가 포함된 데이터를 생성한다. 이 원본 데이터들을 조합하여 의학적으로 의미 있는 새로운 변수를 만든다.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

data = {
    'Patient_ID': ['P001', 'P002', 'P003', 'P004', 'P005'],
    'Height_cm': [175, 160, 180, 165, 150],
    'Weight_kg': [70, 50, 95, 80, 45],
    'Birth_Date': ['1990-05-15', '1982-11-20', '1975-03-10', '2000-01-05', '1955-08-30'],
    'Admission_Date': ['2023-01-01', '2023-02-10', '2023-03-05', '2023-01-20', '2023-04-01'],
    'Discharge_Date': ['2023-01-05', '2023-02-12', '2023-03-20', '2023-02-10', '2023-04-05']
}

df = pd.DataFrame(data)

# 날짜 컬럼은 문자열(String)로 되어있으므로 datetime 객체로 변환해야 계산이 가능하다.
df['Birth_Date'] = pd.to_datetime(df['Birth_Date'])
df['Admission_Date'] = pd.to_datetime(df['Admission_Date'])
df['Discharge_Date'] = pd.to_datetime(df['Discharge_Date'])

print("데이터 준비 완료:")
display(df.head(2))

데이터 준비 완료:


,Patient_ID,Height_cm,Weight_kg,Birth_Date,Admission_Date,Discharge_Date
0,P001,175,70,1990-05-15,2023-01-01,2023-01-05
1,P002,160,50,1982-11-20,2023-02-10,2023-02-12


2. 수치 연산을 통한 변수 생성: BMI 계산
BMI(Body Mass Index)는 체중(kg) / 키(m)^2 로 계산한다. 데이터의 키는 cm 단위이므로 m 단위로 변환이 필요하다.

In [2]:
# 1. 키를 cm에서 m로 변환
df['Height_m'] = df['Height_cm'] / 100

# 2. BMI 공식 적용
df['BMI'] = df['Weight_kg'] / (df['Height_m'] ** 2)

# 3. 소수점 둘째 자리에서 반올림
df['BMI'] = df['BMI'].round(2)

print("BMI 컬럼 생성 완료:")
display(df[['Patient_ID', 'Height_cm', 'Weight_kg', 'BMI']])

BMI 컬럼 생성 완료:


,Patient_ID,Height_cm,Weight_kg,BMI
0,P001,175,70,22.86
1,P002,160,50,19.53
2,P003,180,95,29.32
3,P004,165,80,29.38
4,P005,150,45,20.00


3. 날짜 연산을 통한 변수 생성: 재원 기간(LOS) 및 나이
시계열이 아니더라도, '기간'이나 '나이'는 두 시점의 차이(Diff)로 계산되는 비시계열적 변수로 자주 사용된다.

In [3]:
# 1. 재원 기간 (Length Of Stay, LOS) 계산
# 퇴원일 - 입원일 = Timedelta 객체 (일수 + 시간)
# .dt.days 를 사용하여 '일수(Day)'만 숫자로 추출한다.
df['LOS_Days'] = (df['Discharge_Date'] - df['Admission_Date']).dt.days

print("\n재원 기간(LOS) 계산 완료:")
display(df[['Patient_ID', 'Admission_Date', 'Discharge_Date', 'LOS_Days']])

# 2. 만나이 계산 (현재 연도 기준)
current_year = 2025 # 현재 연도 가정
df['Age'] = current_year - df['Birth_Date'].dt.year

print("\n나이(Age) 계산 완료:")
display(df[['Patient_ID', 'Birth_Date', 'Age']])


재원 기간(LOS) 계산 완료:


,Patient_ID,Admission_Date,Discharge_Date,LOS_Days
0,P001,2023-01-01,2023-01-05,4
1,P002,2023-02-10,2023-02-12,2
2,P003,2023-03-05,2023-03-20,15
3,P004,2023-01-20,2023-02-10,21
4,P005,2023-04-01,2023-04-05,4



나이(Age) 계산 완료:


,Patient_ID,Birth_Date,Age
0,P001,1990-05-15,35
1,P002,1982-11-20,43
2,P003,1975-03-10,50
3,P004,2000-01-05,25
4,P005,1955-08-30,70


4. 조건에 따른 범주화 (Categorization)
연속형 변수(숫자)를 구간별로 나누어 범주형 변수(등급)로 만든다.
예: BMI 수치에 따라 저체중/정상/비만으로 분류한다.

In [4]:
# 함수 정의: BMI 수치에 따라 상태를 반환
def classify_bmi(bmi):
    if bmi < 18.5:
        return 'Underweight'
    elif 18.5 <= bmi < 25.0:
        return 'Normal'
    else:
        return 'Obese'

# apply 함수를 사용하여 데이터프레임 전체에 적용
df['BMI_Class'] = df['BMI'].apply(classify_bmi)

print("\nBMI 등급 분류 완료:")
display(df[['Patient_ID', 'BMI', 'BMI_Class']])


BMI 등급 분류 완료:


,Patient_ID,BMI,BMI_Class
0,P001,22.86,Normal
1,P002,19.53,Normal
2,P003,29.32,Obese
3,P004,29.38,Obese
4,P005,20.00,Normal
